In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

In [2]:
#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

In [3]:
class RSI_Overbought(bt.Indicator):
    lines = ('overbought',)
    
    params = (
        ('level', 70),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) > self.p.level

In [4]:
# Create a Stratey to trade hv
class IVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
#         self.sma_short = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
#         self.sma_long = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
#         self.dsma = self.sma_short - self.sma_long
#         self.mom = bt.Cmp(self.sma_short, self.sma_long)
        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
#         self.rsi_oversold = RSI_Oversold(self.stock_px)
#         self.rsi_overbought = RSI_Overbought(self.stock_px)
        self.rsi_oversold = bt.indicators.RSI(self.stock_px) < 30
        self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70

#         self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
#         self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        self.flag = 0   # -1 sell, 0 no pos, 1 buy.
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size

        # If not in the trade
        if (self.flag == 0):
            if (self.rsi_oversold[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("Oversold indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = 1
            elif (self.rsi_overbought[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("Overbought indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = -1
        
        # If in the long position
        elif (self.flag == 1):
            # Close the position if the SELL signal triggered or reached the max days to hold
            if (self.rsi_overbought[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.rsi_oversold[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                print("In position %d days" % self.days_in_trade)
        
        # If in the short position
        elif (self.flag == -1):
            # Close the position if the BUY signal triggered or reached the max days to hold
            if (self.rsi_oversold[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
                self.close(exectype=bt.Order.Close)
                print("Closing the position")
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.rsi_overbought[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                print("In position %d days" % self.days_in_trade)
                
#         print("Flag = %d" % self.flag)
#         print("In position %d days" % self.days_in_trade)
            
        
        # Check if we are currently in a trade
#         if(not size):
#             #If not has our indicator triggered?
#             if self.rsi_oversold[0] > 0:
#                 print("Oversold indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
                
#             if self.rsi_overbought[0] > 0:
#                 print("Overbought indicator triggered...entering position")
#                 self.sell(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
                               
#         else:
#             self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            
#             if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0

In [6]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('MSFT.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = 'iv30' #iv30
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(IVStrategy, symbols=symbol_cols, y=y)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='iv30-strategy-MSFT-rsi-1.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2010-01-04, Close, 30.95; Vol, 21.37
2010-01-05, Close, 30.93; Vol, 21.53
2010-01-06, Close, 30.75; Vol, 22.06
2010-01-07, Close, 30.45; Vol, 23.36
2010-01-08, Close, 30.68; Vol, 22.58
2010-01-11, Close, 30.24; Vol, 25.33
2010-01-12, Close, 30.03; Vol, 26.48
2010-01-13, Close, 30.37; Vol, 25.65
2010-01-14, Close, 31.01; Vol, 26.00
2010-01-15, Close, 30.90; Vol, 23.56
2010-01-19, Close, 31.10; Vol, 24.63
2010-01-20, Close, 30.61; Vol, 25.76
2010-01-21, Close, 30.06; Vol, 27.71
2010-01-22, Close, 28.95; Vol, 31.68
2010-01-25, Close, 29.37; Vol, 33.29
2010-01-26, Close, 29.55; Vol, 32.98
2010-01-27, Close, 29.70; Vol, 32.18
2010-01-28, Close, 29.29; Vol, 35.35
2010-01-29, Close, 28.12; Vol, 26.73
Oversold indicator triggered...entering position
2010-02-01, Close, 28.37; Vol, 24.81
In position 2 days
2010-02-02, Close, 28.47; Vol, 22.73
In position 3 days
2010-02-03, Close, 28.61; Vol, 22.26
In position 4 days
2010-02-04, Close, 27.86; Vol, 26.51
In posi

2011-04-28, Close, 26.74; Vol, 26.07
2011-04-29, Close, 25.85; Vol, 19.30
2011-05-02, Close, 25.54; Vol, 18.88
2011-05-03, Close, 25.80; Vol, 18.46
2011-05-04, Close, 26.15; Vol, 19.28
2011-05-05, Close, 25.76; Vol, 20.16
2011-05-06, Close, 25.88; Vol, 19.17
2011-05-09, Close, 25.85; Vol, 18.31
2011-05-10, Close, 25.48; Vol, 19.48
2011-05-11, Close, 25.28; Vol, 19.23
2011-05-12, Close, 25.31; Vol, 19.19
2011-05-13, Close, 25.06; Vol, 19.16
2011-05-16, Close, 24.68; Vol, 21.03
2011-05-17, Close, 24.52; Vol, 20.57
2011-05-18, Close, 24.67; Vol, 19.62
2011-05-19, Close, 24.70; Vol, 18.72
2011-05-20, Close, 24.49; Vol, 18.16
2011-05-23, Close, 24.18; Vol, 19.86
2011-05-24, Close, 24.20; Vol, 19.79
2011-05-25, Close, 24.26; Vol, 18.55
2011-05-26, Close, 24.76; Vol, 18.26
2011-05-27, Close, 24.79; Vol, 17.77
2011-05-31, Close, 24.98; Vol, 17.58
2011-06-01, Close, 24.44; Vol, 20.63
2011-06-02, Close, 24.23; Vol, 20.25
2011-06-03, Close, 23.93; Vol, 20.31
2011-06-06, Close, 24.13; Vol, 21.24
2

2012-08-24, Close, 30.52; Vol, 16.72
2012-08-27, Close, 30.63; Vol, 17.30
2012-08-28, Close, 30.66; Vol, 17.15
2012-08-29, Close, 30.66; Vol, 17.12
2012-08-30, Close, 30.34; Vol, 18.15
2012-08-31, Close, 30.91; Vol, 13.90
2012-09-04, Close, 30.38; Vol, 20.38
2012-09-05, Close, 30.34; Vol, 19.91
2012-09-06, Close, 31.31; Vol, 18.29
2012-09-07, Close, 30.93; Vol, 18.78
2012-09-10, Close, 30.74; Vol, 19.47
2012-09-11, Close, 30.84; Vol, 19.61
2012-09-12, Close, 30.80; Vol, 19.50
2012-09-13, Close, 30.93; Vol, 19.55
2012-09-14, Close, 31.14; Vol, 19.20
2012-09-17, Close, 31.18; Vol, 19.62
2012-09-18, Close, 31.16; Vol, 19.48
2012-09-19, Close, 31.11; Vol, 19.20
2012-09-20, Close, 31.38; Vol, 19.07
2012-09-21, Close, 31.21; Vol, 18.91
2012-09-24, Close, 30.79; Vol, 20.16
2012-09-25, Close, 30.41; Vol, 20.95
2012-09-26, Close, 30.13; Vol, 21.86
2012-09-27, Close, 30.24; Vol, 21.62
2012-09-28, Close, 29.76; Vol, 21.88
2012-10-01, Close, 29.48; Vol, 22.85
2012-10-02, Close, 29.63; Vol, 22.66
2

2014-06-19, Close, 41.47; Vol, 14.63
2014-06-20, Close, 41.75; Vol, 14.06
2014-06-23, Close, 41.93; Vol, 14.12
2014-06-24, Close, 41.65; Vol, 14.43
2014-06-25, Close, 42.00; Vol, 14.34
2014-06-26, Close, 41.69; Vol, 14.94
2014-06-27, Close, 42.18; Vol, 14.31
2014-06-30, Close, 41.75; Vol, 16.27
2014-07-01, Close, 41.84; Vol, 16.00
2014-07-02, Close, 41.85; Vol, 16.11
2014-07-03, Close, 41.78; Vol, 16.66
2014-07-07, Close, 42.00; Vol, 17.52
2014-07-08, Close, 41.88; Vol, 20.51
2014-07-09, Close, 41.70; Vol, 19.72
2014-07-10, Close, 41.75; Vol, 19.58
2014-07-11, Close, 41.98; Vol, 19.62
2014-07-14, Close, 42.09; Vol, 19.31
2014-07-15, Close, 42.26; Vol, 19.48
2014-07-16, Close, 44.00; Vol, 21.23
Overbought indicator triggered...entering position
2014-07-17, Close, 44.42; Vol, 23.19
In position 2 days
2014-07-18, Close, 44.50; Vol, 20.65
In position 3 days
2014-07-21, Close, 44.84; Vol, 26.07
In position 4 days
2014-07-22, Close, 44.81; Vol, 25.02
In position 5 days
2014-07-23, Close, 44.

2016-04-27, Close, 50.88; Vol, 19.53
2016-04-28, Close, 50.08; Vol, 19.36
2016-04-29, Close, 49.90; Vol, 20.65
Oversold indicator triggered...entering position
2016-05-02, Close, 50.65; Vol, 18.91
In position 2 days
2016-05-03, Close, 49.79; Vol, 20.50
In position 3 days
2016-05-04, Close, 49.97; Vol, 20.74
In position 4 days
2016-05-05, Close, 49.90; Vol, 20.37
In position 5 days
2016-05-06, Close, 50.24; Vol, 19.29
In position 6 days
2016-05-09, Close, 50.16; Vol, 19.68
In position 7 days
2016-05-10, Close, 51.02; Vol, 19.01
2016-05-11, Close, 51.09; Vol, 18.91
2016-05-12, Close, 51.67; Vol, 18.45
2016-05-13, Close, 51.18; Vol, 18.61
2016-05-16, Close, 51.88; Vol, 18.12
2016-05-17, Close, 51.16; Vol, 18.86
2016-05-18, Close, 50.58; Vol, 19.38
2016-05-19, Close, 50.22; Vol, 20.14
2016-05-20, Close, 50.57; Vol, 18.64
2016-05-23, Close, 50.21; Vol, 18.81
2016-05-24, Close, 51.62; Vol, 17.78
2016-05-25, Close, 52.17; Vol, 16.86
2016-05-26, Close, 51.93; Vol, 16.74
2016-05-27, Close, 52.1

2017-12-04, Close, 81.08; Vol, 19.79
2017-12-05, Close, 81.59; Vol, 19.07
2017-12-06, Close, 82.78; Vol, 15.83
2017-12-07, Close, 82.49; Vol, 16.62
2017-12-08, Close, 84.16; Vol, 14.46
2017-12-11, Close, 85.23; Vol, 15.73
2017-12-12, Close, 85.58; Vol, 15.11
2017-12-13, Close, 85.35; Vol, 15.46
2017-12-14, Close, 84.69; Vol, 15.20
2017-12-15, Close, 86.85; Vol, 14.90
2017-12-18, Close, 86.38; Vol, 14.88
2017-12-19, Close, 85.83; Vol, 15.24
2017-12-20, Close, 85.52; Vol, 14.94
2017-12-21, Close, 85.50; Vol, 13.90
2017-12-22, Close, 85.51; Vol, 14.02
2017-12-26, Close, 85.40; Vol, 15.31
2017-12-27, Close, 85.71; Vol, 14.88
2017-12-28, Close, 85.72; Vol, 14.15
2017-12-29, Close, 85.54; Vol, 14.57
2018-01-02, Close, 85.95; Vol, 19.25
2018-01-03, Close, 86.35; Vol, 20.72
2018-01-04, Close, 87.11; Vol, 22.29
2018-01-05, Close, 88.19; Vol, 21.95
Overbought indicator triggered...entering position
2018-01-08, Close, 88.28; Vol, 22.21
In position 2 days
2018-01-09, Close, 88.22; Vol, 22.30
In po

Final Portfolio Value: 99957.99